Few key observations
- If both and non-encrypted files are base-32 encoded, accuracy drops really low.
   * Even in this case, fourier analysis (specifically autocorrelation and mean) provide good discrminators
   * Renyi's entropy at alpha=infinity gives good discrimination
   * Overall we see around 13% improvement in F1-score
- Specifically for webpfiles
   * We see that higher moments of around 14 gives a good discrimination
   * Fourier analysis still gives benefits here
- For webpfiles where both unencrypted and encrypted content are base32 encrypted
   * Fourier analysis is still beneficial
- For Non-webp files where plaintext is not base-32 encrypted
   * Accuracy is decent with just the base parameters, but with fourier analysis, we can close to perfect detection


# Contents
- ## [Implementation](#mplementation)
 - ### [List of Files](#list-of-files)
 - ### [Utility to call gc](#gc)
 - ### [Selection of Columns](#select-columns)
 - ### [Function to compare the three selections of columns for a given dataset](#compare-fn)
 - ### [Load the datasets](#load-dataset)
 - ### [Select the datasets](#dataset-selection)
 - ### [Run all combinations of data](#dataset-selection)
- ## [Results](#results)
 - ### [Print the raw results](#raw-results)
 - ### [Sort by the improvement when using Fourier transforms over baseline](#sorted-fourier)
 - ### [Measure improvement when only webp files are used](#webp-only)
 - ### [Measure improvement when webp files are not present](#no-webp)
 - ### [Measure improvement for balanced datasets](#balanced-dataset)
 - ### [Measure the improvement when all files are used](#full-dataset)
 


## Implementation <a class="anchor" id="implementation">
    
[top](#Contents)

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

from functools import lru_cache

import gc

import sklearn
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score, accuracy_score

import matplotlib.pyplot as plt
import glob

plt.rcParams["figure.figsize"] = (20,20)

### List of files <a class="anchor" id="list-of-files">

[top](#Contents)

In [2]:
!ls

expanded.plaintext.base32.csv.gz
expanded.pyencrypted_v1.b32.csv.gz
expanded.pyencrypted_v1.csv.gz
expanded.pyencrypted_v2.base32.csv.gz
expanded.pyencrypted_v2.csv.gz
iteration_2.1_compare_base32_alternate_encryption_scheme.ipynb
iteration_2.1_compare_base32_logistic_regression.ipynb
plaintext.base32.combined.csv.gz
plaintext.combined.csv.gz
plaintext.expanded.csv.gz


### Utility to call GC <a class="anchor" id="gc">
    
[top](#Contents)

In [3]:
def call_gc():
    for i in range(3):
        for j in range(3):
            gc.collect(j)

### Selection of columns <a class="anchor" id="select-columns">
    
This function is used to select the columns.
There are three different configurations:
    
    1. Baseline - statistics like autocorrelation, shannon entropy, chi-square etc. are used
    
    2. Advanced - More statistics like moments, Tsallis and Renyi entropy, higher moments etc.
    
    3. Fourier - Some statistics of the Fourier Power Spectrum distribution
    
[top](#Contents)

In [4]:
def get_columns(thisdf):
    baseline_columns = [c for c in thisdf.columns if c.startswith('baseline') and "head" not in c and "tail" not in c]
    baseline_columns = [c for c in baseline_columns if "filesize" not in c]
    baseline_columns = [c for c in baseline_columns if "begin" not in c and "end" not in c]

    advanced_columns = [c for c in thisdf.columns if "advanced" in c]
    advanced_columns = [c for c in advanced_columns if "begin" not in c and "end" not in c]
    advanced_columns = [c for c in advanced_columns if "head" not in c and "tail" not in c]
    advanced_columns = [c for c in advanced_columns if "start" not in c]
    advanced_columns = list(set(advanced_columns + baseline_columns))

    fourier_columns = [c for c in thisdf.columns if "fourier" in c and "value" not in c]
    fourier_columns = [c for c in fourier_columns if "1byte" in c]
    fourier_columns = [c for c in fourier_columns if "begin" not in c and "end" not in c]
    fourier_columns = [c for c in fourier_columns if "head" not in c and "tail" not in c]
    fourier_columns = [c for c in fourier_columns if "start" not in c]
    fourier_columns = list(set(advanced_columns + fourier_columns))
    
    return {\
        "baseline": baseline_columns,\
        "advanced": advanced_columns,\
        "fourier": fourier_columns,\
    }


### Function to compare the three selections of columns for a given dataset <a class="anchor" id="compare-fn">
    
For the three different sets of measurements described above, compare all the three sets for their f1 score
    
[top](#Contents)

In [5]:
def compare_feature_selections(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    columns_dict = get_columns(X)
    ret = dict()
    for name, collist in columns_dict.items():
        XX_train, XX_test = X_train[collist], X_test[collist]
        #rfc = RandomForestClassifier(n_jobs=10, random_state=42)
        lr = LogisticRegression(n_jobs=10, random_state=42, max_iter=5000)
        estimators = [('std,', MinMaxScaler()), ('LogisticRegression', lr)]
        pipeline = Pipeline(estimators)
        pipeline.fit(XX_train, y_train)
        y_pred = pipeline.predict(XX_test)
        f1 = f1_score(y_test, y_pred)
        ret[f"{name}-f1_score"] = f1
    return ret

def compare_feature_selections_wrapper(data):
    X = data[[c for c in data.columns if "is_encrypted" != c]]
    y = data["is_encrypted"]
    return compare_feature_selections(X, y)

### Load the datasets <a class="anchor" id="load-dataset">

We don't want to load the datasets again and again, so we load them once.
The LRU cache ensures that the next time this is called, we don't load the datasets again.
    
[top](#Contents)

In [6]:
@lru_cache
def load_datasets_once():
    """Load all datasets only once
    
    We want to load the datasets only once. Once loaded
    serve from cache
    """
    datasets = dict()
    for file in glob.glob("*.csv.gz"):
        df = pd.read_csv(file)
        df = df.sample(frac=1).reset_index(drop=True)
        df["is_encrypted"] = 1 if "encr" in file.lower() else 0
        datasets[file] = df
    return datasets

### Select the datasets <a class="anchor" id="dataset-selection">

Select the dataset, combine it, shuffle it and return
    
We used several schemes:
* Plaintext
  - The base plaintext dataset of around 50k files
  - Expanded dataset, where all files that were less than 1500 bytes were expanded by repeating the file
  - Base32 encoding of
   - original plaintext
   - expanded plaintext
* Encryption : We used two schemes for encruption
  - v1 : AES 256 encryption of the full file, again two varieties were used
    - Plain encryption
    - Encryption followed by base32 encoding
  - v2 : AES encryption of the file, the first and the last 128 bytes are not encrypted. Of the remaining file, every alternate 16 bytes are encrypted.
    - Plain encryption
    - encryption followed by base32 encoding
    
We select combinations of the above datasets
    
Furthermore, we add further combinations by including and excluding webp files, and including and excluding non-webp files.
    
[top](#Contents)

In [7]:
def get_dataset(\
               exclude_plaintext_nonbase32=False,
               exclude_plaintext_base32=False,
               exclude_encrypted_v1=False,
               exclude_encrypted_v2=False,
               exclude_encrypted_base32=False,
               exclude_encrypted_nonbase32=False,
               exclude_webp=False,
               exclude_nonwebp=False):
    description = {k:v for k,v in locals().items() if "exclude" in k}
    files = glob.glob("*.csv.gz")
    
    
    if exclude_plaintext_nonbase32:
        files = [f for f in files if \
                     "plaintext" not in f \
                     or "_base32" in f \
                     or "_b32" in f]
    if exclude_plaintext_base32:
        files = [f for f in files if\
                    "plaintext" not in f or \
                    ("base32" not in f and "b32" not in f)]
    if exclude_encrypted_v1:
        files = [f for f in files if\
                    "_v1" not in f or "encr" not in f]
    if exclude_encrypted_v2:
        files = [f for f in files if \
                    "_v2" not in f or "encr" not in f]
    if exclude_encrypted_base32:
        files = [ f for f in files if\
                    "encr" not in f or\
                    ("base32" not in f and "b32" not in f)]
    if exclude_encrypted_nonbase32:
        files = [f for f in files if \
                    "encr" not in f or \
                    "base32" in f or "b32" in f]
        
        
    encrypted_count = len([f for f in files if "encr" in f])
    non_encrypted_count = len([f for f in files if "encr" not in f])
    
    if 0 == encrypted_count or 0 == non_encrypted_count or 0 == len(files):
        return description, None, -1, -1
    
    all_dfs = list()
    all_loaded_datasets = load_datasets_once()
    for filename in files:
        all_dfs.append(all_loaded_datasets[filename])
    
    combined_df = pd.concat(all_dfs)
    
    if exclude_webp:
        combined_df = combined_df[combined_df["extended.extension"] != ".webp"]
    if exclude_nonwebp:
        combined_df = combined_df[combined_df["extended.extension"] == ".webp"]
        
    if len(combined_df) == 0:
        return description, None, -1, -1
    
        
    encrypt_count = len(combined_df[combined_df["is_encrypted"] == 1])
    nonencrypted_count = len(combined_df[combined_df["is_encrypted"] == 0])

    
    if 0 == len(combined_df) or 0 == encrypt_count or 0 == nonencrypted_count:
        return description, None, -1, -1
    
    return description, combined_df, encrypt_count, nonencrypted_count

### Run all combinations of data <a class="anchor" id="dataset-selection">

For all combinations of the datasets, compare the three classes (basic, advanced, and Fourier)
    
[top](#Contents)

In [8]:
all_results = []
count = 2
try:
    for a in [True, False]:
        for b in [True, False]:
            for c in [True, False]:
                for k in [True, False]:
                    for e in [True, False]:
                        for f in [True, False]:
                            for g in [True, False]:
                                for h in [True, False]:
                                    desc, df, ec, nec = get_dataset(a, b, c, k, e, f, g, h)
                                    if df is not None:
                                        result = compare_feature_selections_wrapper(df)
                                        call_gc()
                                        result = {**desc, **result}
                                        result["n_encrypted"] = ec
                                        result["n_non_encrypted"] = nec
                                        result["ratio_encrypt_nonencrypt"] = float(ec) / nec
                                        all_results.append(result)

                                        """
                                        count -= 1
                                        if count == 0:
                                            # A non-standard way to break out of 
                                            # all nested loops
                                            raise Exception("breaking out")
                                        """
except Exception as e:
    print(e)

df_dict = {k:[] for k in all_results[0]}
for result in all_results:
    for k, v in result.items():
        df_dict[k].append(v)

df = pd.DataFrame(df_dict)
df["best_f1_score"] = df[[c for c in df.columns if "f1_score" in c]].max(axis=1)

df["improvement_in_advanced"] = df["advanced-f1_score"] - df["baseline-f1_score"]
df["improvement_in_fourier"] = df["fourier-f1_score"] - df["baseline-f1_score"]
df = df.sort_values(by="improvement_in_fourier", ascending=False)
    
                                

## Results <a class="anchor" id="results">
    
In this section, we present the results
    
[top](#Contents)

### Print the raw results <a class="anchor" id="raw-results">
    
[top](#Contents)

In [9]:
df

,exclude_plaintext_nonbase32,exclude_plaintext_base32,exclude_encrypted_v1,exclude_encrypted_v2,exclude_encrypted_base32,exclude_encrypted_nonbase32,exclude_webp,exclude_nonwebp,baseline-f1_score,advanced-f1_score,fourier-f1_score,n_encrypted,n_non_encrypted,ratio_encrypt_nonencrypt,best_f1_score,improvement_in_advanced,improvement_in_fourier
39,False,False,False,True,False,True,True,False,0.004574,0.900238,0.921809,49228,196912,0.25,0.921809,0.895663,0.917235
41,False,False,False,True,False,True,False,False,0.005320,0.712470,0.761684,80853,323412,0.25,0.761684,0.707150,0.756364
35,False,False,True,False,False,False,False,False,0.000000,0.284363,0.477682,161706,323412,0.50,0.477682,0.284363,0.477682
10,False,True,False,True,True,False,False,True,0.166630,0.605710,0.625647,31625,63250,0.50,0.625647,0.439080,0.459018
27,False,False,True,False,True,False,True,False,0.466882,0.615413,0.908005,49228,196912,0.25,0.908005,0.148532,0.441124
0,False,True,True,False,True,False,True,False,0.471024,0.606337,0.908055,49228,98456,0.50,0.908055,0.135312,0.437030
2,False,True,True,False,True,False,False,False,0.313597,0.493990,0.727650,80853,161706,0.50,0.727650,0.180393,0.414053
29,False,False,True,False,True,False,False,False,0.317690,0.500931,0.729395,80853,323412,0.25,0.729395,0.183242,0.411705
30,False,False,True,False,False,True,True,False,0.000157,0.376067,0.398670,49228,196912,0.25,0.398670,0.375911,0.398513
37,False,False,False,True,True,False,False,True,0.150285,0.542275,0.547251,31625,126500,0.25,0.547251,0.391989,0.396965


In [10]:
df.to_csv("../comparison.csv")

In [11]:
df = df.sort_values(by="improvement_in_fourier", ascending=False)

### Sort by the improvement when using Fourier transforms over baseline <a class="anchor" id="sorted-fourier">

We also take a few other things:
* Room for improvement (1 - baseline score)
* How much of the room for improvement was filled by fourier transforms (percentage)
* Difference between F1 score with advanced and fourier
        
[top](#Contents)

In [12]:
df["improvement_in_advanced"] = df["advanced-f1_score"] - df["baseline-f1_score"]
df["room_for_improvement"] = 1.0 - df["baseline-f1_score"]
df["fourier_minus_advanced"] = df["fourier-f1_score"] - df["advanced-f1_score"]
df["percentage_of_room_filled"] = df["improvement_in_fourier"] * 100.0 / df["room_for_improvement"]
#df.sort_values(by="fourier_minus_advanced", ascending=False)
df

,exclude_plaintext_nonbase32,exclude_plaintext_base32,exclude_encrypted_v1,exclude_encrypted_v2,exclude_encrypted_base32,exclude_encrypted_nonbase32,exclude_webp,exclude_nonwebp,baseline-f1_score,advanced-f1_score,fourier-f1_score,n_encrypted,n_non_encrypted,ratio_encrypt_nonencrypt,best_f1_score,improvement_in_advanced,improvement_in_fourier,room_for_improvement,fourier_minus_advanced,percentage_of_room_filled
39,False,False,False,True,False,True,True,False,0.004574,0.900238,0.921809,49228,196912,0.25,0.921809,0.895663,0.917235,0.995426,0.021572,92.145015
41,False,False,False,True,False,True,False,False,0.005320,0.712470,0.761684,80853,323412,0.25,0.761684,0.707150,0.756364,0.994680,0.049214,76.040950
35,False,False,True,False,False,False,False,False,0.000000,0.284363,0.477682,161706,323412,0.50,0.477682,0.284363,0.477682,1.000000,0.193319,47.768225
10,False,True,False,True,True,False,False,True,0.166630,0.605710,0.625647,31625,63250,0.50,0.625647,0.439080,0.459018,0.833370,0.019937,55.079670
27,False,False,True,False,True,False,True,False,0.466882,0.615413,0.908005,49228,196912,0.25,0.908005,0.148532,0.441124,0.533118,0.292592,82.744027
0,False,True,True,False,True,False,True,False,0.471024,0.606337,0.908055,49228,98456,0.50,0.908055,0.135312,0.437030,0.528976,0.301718,82.618234
2,False,True,True,False,True,False,False,False,0.313597,0.493990,0.727650,80853,161706,0.50,0.727650,0.180393,0.414053,0.686403,0.233660,60.322105
29,False,False,True,False,True,False,False,False,0.317690,0.500931,0.729395,80853,323412,0.25,0.729395,0.183242,0.411705,0.682310,0.228464,60.339896
30,False,False,True,False,False,True,True,False,0.000157,0.376067,0.398670,49228,196912,0.25,0.398670,0.375911,0.398513,0.999843,0.022602,39.857558
37,False,False,False,True,True,False,False,True,0.150285,0.542275,0.547251,31625,126500,0.25,0.547251,0.391989,0.396965,0.849715,0.004976,46.717488


### Measure improvement when only webp files are used <a class="anchor" id="webp-only">

We want to measure this alone because in the literature, poor performance in webp files has been mentioned
    
[top](#Contents)

In [13]:
df[(df["exclude_webp"] == False) & (df["exclude_nonwebp"] == True)]

,exclude_plaintext_nonbase32,exclude_plaintext_base32,exclude_encrypted_v1,exclude_encrypted_v2,exclude_encrypted_base32,exclude_encrypted_nonbase32,exclude_webp,exclude_nonwebp,baseline-f1_score,advanced-f1_score,fourier-f1_score,n_encrypted,n_non_encrypted,ratio_encrypt_nonencrypt,best_f1_score,improvement_in_advanced,improvement_in_fourier,room_for_improvement,fourier_minus_advanced,percentage_of_room_filled
10,False,True,False,True,True,False,False,True,0.166630,0.605710,0.625647,31625,63250,0.50,0.625647,0.439080,0.459018,0.833370,0.019937,55.079670
37,False,False,False,True,True,False,False,True,0.150285,0.542275,0.547251,31625,126500,0.25,0.547251,0.391989,0.396965,0.849715,0.004976,46.717488
43,False,False,False,True,False,False,False,True,0.067426,0.353474,0.376496,63250,126500,0.50,0.376496,0.286048,0.309069,0.932574,0.023022,33.141554
31,False,False,True,False,False,True,False,True,0.040325,0.250610,0.335685,31625,126500,0.25,0.335685,0.210285,0.295360,0.959675,0.085075,30.777077
34,False,False,True,False,False,False,False,True,0.154326,0.340892,0.431770,63250,126500,0.50,0.431770,0.186567,0.277444,0.845674,0.090878,32.807487
28,False,False,True,False,True,False,False,True,0.371698,0.479251,0.540740,31625,126500,0.25,0.540740,0.107553,0.169042,0.628302,0.061489,26.904586
1,False,True,True,False,True,False,False,True,0.376303,0.475237,0.541249,31625,63250,0.50,0.541249,0.098934,0.164946,0.623697,0.066012,26.446476
40,False,False,False,True,False,True,False,True,0.029566,0.000752,0.190629,31625,126500,0.25,0.190629,-0.028815,0.161062,0.970434,0.189877,16.596949
19,False,True,False,False,True,False,False,True,0.535105,0.674345,0.679730,63250,63250,1.00,0.679730,0.139240,0.144625,0.464895,0.005385,31.109214
46,False,False,False,False,True,False,False,True,0.526355,0.663859,0.662163,63250,126500,0.50,0.663859,0.137504,0.135807,0.473645,-0.001697,28.672798


### Measure improvement when webp files are not present <a class="anchor" id="no-webp">
    
[top](#Contents)

In [14]:
df[(df["exclude_webp"] == True) & (df["exclude_nonwebp"] == False)]

,exclude_plaintext_nonbase32,exclude_plaintext_base32,exclude_encrypted_v1,exclude_encrypted_v2,exclude_encrypted_base32,exclude_encrypted_nonbase32,exclude_webp,exclude_nonwebp,baseline-f1_score,advanced-f1_score,fourier-f1_score,n_encrypted,n_non_encrypted,ratio_encrypt_nonencrypt,best_f1_score,improvement_in_advanced,improvement_in_fourier,room_for_improvement,fourier_minus_advanced,percentage_of_room_filled
39,False,False,False,True,False,True,True,False,0.004574,0.900238,0.921809,49228,196912,0.25,0.921809,0.895663,0.917235,0.995426,0.021572,92.145015
27,False,False,True,False,True,False,True,False,0.466882,0.615413,0.908005,49228,196912,0.25,0.908005,0.148532,0.441124,0.533118,0.292592,82.744027
0,False,True,True,False,True,False,True,False,0.471024,0.606337,0.908055,49228,98456,0.50,0.908055,0.135312,0.437030,0.528976,0.301718,82.618234
30,False,False,True,False,False,True,True,False,0.000157,0.376067,0.398670,49228,196912,0.25,0.398670,0.375911,0.398513,0.999843,0.022602,39.857558
33,False,False,True,False,False,False,True,False,0.267677,0.428018,0.649408,98456,196912,0.50,0.649408,0.160340,0.381731,0.732323,0.221390,52.126052
42,False,False,False,True,False,False,True,False,0.587016,0.925286,0.920249,98456,196912,0.50,0.925286,0.338270,0.333233,0.412984,-0.005037,80.689150
6,False,True,True,False,False,False,True,False,0.682626,0.812982,0.948674,98456,98456,1.00,0.948674,0.130356,0.266048,0.317374,0.135692,83.828017
18,False,True,False,False,True,False,True,False,0.775331,0.823511,0.950414,98456,98456,1.00,0.950414,0.048179,0.175082,0.224669,0.126903,77.929208
45,False,False,False,False,True,False,True,False,0.774707,0.824627,0.948783,98456,196912,0.50,0.948783,0.049920,0.174076,0.225293,0.124155,77.266297
24,False,True,False,False,False,False,True,False,0.836775,0.910909,0.972644,196912,98456,2.00,0.972644,0.074134,0.135869,0.163225,0.061735,83.240297


### Measure improvement for balanced datasets <a class="anchor" id="balanced-dataset">

Most of our dataset selections are not balanced, and we didn't do anything to mitigate
unablanced datasets. This is because we are measuring the baseline and improvement on the
same dataset.
    
However, we will select those instances where the datasets were balanced to see if the
improvement is still significant in these instances.
    
[top](#Contents)

In [15]:

df[df["ratio_encrypt_nonencrypt"] == 1.0]

,exclude_plaintext_nonbase32,exclude_plaintext_base32,exclude_encrypted_v1,exclude_encrypted_v2,exclude_encrypted_base32,exclude_encrypted_nonbase32,exclude_webp,exclude_nonwebp,baseline-f1_score,advanced-f1_score,fourier-f1_score,n_encrypted,n_non_encrypted,ratio_encrypt_nonencrypt,best_f1_score,improvement_in_advanced,improvement_in_fourier,room_for_improvement,fourier_minus_advanced,percentage_of_room_filled
6,False,True,True,False,False,False,True,False,0.682626,0.812982,0.948674,98456,98456,1.0,0.948674,0.130356,0.266048,0.317374,0.135692,83.828017
8,False,True,True,False,False,False,False,False,0.639129,0.774686,0.865153,161706,161706,1.0,0.865153,0.135557,0.226024,0.360871,0.090467,62.632852
18,False,True,False,False,True,False,True,False,0.775331,0.823511,0.950414,98456,98456,1.0,0.950414,0.048179,0.175082,0.224669,0.126903,77.929208
19,False,True,False,False,True,False,False,True,0.535105,0.674345,0.679730,63250,63250,1.0,0.679730,0.139240,0.144625,0.464895,0.005385,31.109214
52,False,False,False,False,False,False,False,True,0.497310,0.636826,0.626360,126500,126500,1.0,0.636826,0.139516,0.129050,0.502690,-0.010466,25.671964
16,False,True,False,True,False,False,False,True,0.689850,0.802971,0.803380,63250,63250,1.0,0.803380,0.113121,0.113530,0.310150,0.000409,36.604967
51,False,False,False,False,False,False,True,False,0.736074,0.785781,0.840297,196912,196912,1.0,0.840297,0.049707,0.104223,0.263926,0.054516,39.489649
20,False,True,False,False,True,False,False,False,0.727686,0.726249,0.809966,161706,161706,1.0,0.809966,-0.001436,0.082281,0.272314,0.083717,30.215307
7,False,True,True,False,False,False,False,True,0.731170,0.782615,0.802375,63250,63250,1.0,0.802375,0.051446,0.071205,0.268830,0.019760,26.487097
15,False,True,False,True,False,False,True,False,0.942648,0.975480,0.990567,98456,98456,1.0,0.990567,0.032832,0.047919,0.057352,0.015087,83.552487


### Measure the improvement when all files are used <a class="anchor" id="full-dataset">

In most of our dataset selections, we excluded some data points based on our criteria.

In this, we measure the improvement when we use all data points
    
[top](#Contents)

In [16]:
# Measure everything for the full dataset
df2 = df
for colname in df2.columns:
    if "exclude" in colname:
        df2 = df2[df2[colname] == False]
df2

,exclude_plaintext_nonbase32,exclude_plaintext_base32,exclude_encrypted_v1,exclude_encrypted_v2,exclude_encrypted_base32,exclude_encrypted_nonbase32,exclude_webp,exclude_nonwebp,baseline-f1_score,advanced-f1_score,fourier-f1_score,n_encrypted,n_non_encrypted,ratio_encrypt_nonencrypt,best_f1_score,improvement_in_advanced,improvement_in_fourier,room_for_improvement,fourier_minus_advanced,percentage_of_room_filled
53,False,False,False,False,False,False,False,False,0.683715,0.686916,0.728539,323412,323412,1.0,0.728539,0.003201,0.044824,0.316285,0.041623,14.171939


In [17]:
df.to_csv("../comparison_logistic_regression_final.csv")